# Corpus Modification

In [41]:
import json
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage
from stop_words import get_stop_words
import seaborn as sns

from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import LabelEncoder

from utils import add_epoch_division, linkage_matrix, plot_dendrogram, remove_noise_poet, text_cleaning

In [42]:
corpus = text_cleaning(pd.read_csv("../corpora/german_poems.csv"))

In [44]:
with open("epochs.json") as f:
    epochs = json.loads(f.read())

epochs = epochs["brenner"]
epoch_exception = "Klassik_Romantik"
corpus = add_epoch_division(corpus, epochs, epoch_exception=epoch_exception)

In [4]:
LOWERCASE = True
MAX_FEATURES = 10000
STOP_WORDS = get_stop_words("de")

In [69]:
def merge_corpus_poets(corpus, min_count=6):
    """ Merge poems in corpus by poet. Epoch with the most entries will be chosen.
    """
    df = corpus.copy()
    poets = [k for k, v in dict(df.poet.value_counts()).items() if v >= min_count]
    df = df[df.poet.isin(poets)]
    new_poems = {}

    for idx, poet in enumerate(list(np.unique(df.poet))):
        pcorpus = df[df.poet == poet]
        epochs = dict(pcorpus.epoch.value_counts())
        s = " ".join(pcorpus.poem)
        new_poems[idx] = [idx, poet, s, max(epochs)]
        
    mod_c = pd.DataFrame.from_dict(new_poems).T
    mod_c.columns = ["id", "poet", "poem", "epoch"]
    
    return mod_c

In [70]:
merge_corpus_poets(corpus)

,id,poet,poem,epoch
0,0,"Abschatz, Hans Assmann von",Du angenehmer Haeyn voll stiller Einsamkeiten/...,Frühaufklärung
1,1,"Abschatz, Hans Aßmann von",Was bleibt von Reichen unbekant? Hält gleich d...,Barock
2,2,"Ahlefeld, Charlotte von","Nimm hin dies Bild, das auch in weite Ferne Di...",Biedermeier
3,3,Angelus Silesius,Der Liebe Mittelpunkt ist Gott und auch ihr Kr...,Barock
4,4,"Arndt, Ernst Moritz",Wo kommst du her in dem roten Kleid Und färbst...,
...,...,...,...,...
228,228,"Zesen, Philipp von",Echo zeige mier mein Leben/ E. ja dier eben. W...,Barock
229,229,"Ziegler, Christiana Mariana von","Die Dicht- und Redner Kunst liebt, was der Wit...",Frühaufklärung
230,230,"Zinzendorf, Nicolaus Ludwig von","Brueder, last uns Jhn erheben, Den ihr ohne Na...",Frühaufklärung
231,231,"Zinzendorf, Nikolaus Ludwig von","Du fragst: Du rechtes Witwen-Herz, Ich sage vo...",Frühaufklärung


In [57]:
mod_c.epoch.value_counts()

Realismus            71
Frühaufklärung       44
Aufklärung           43
Barock               42
Biedermeier          41
Moderne              15
                      7
Weimarer_Republik     2
Name: epoch, dtype: int64